# Set up

### lichess database: https://database.lichess.org/

In [ ]:
!wget --no-verbose https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst

2023-03-21 22:07:24 URL:https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst [17761302/17761302] -> "lichess_db_standard_rated_2013-01.pgn.zst" [1]


In [ ]:
!apt-get -q install zstd

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,592 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 zstd amd64 1.4.4+dfsg-3ubuntu0.1 [343 kB]
Fetched 343 kB in 1s (604 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 128276 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb ...
Unpacking zstd (1.4.4+dfsg-3ubuntu0.1) ...
Setting up zstd (1.4.4+dfsg-3ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
!unzstd lichess_db_standard_rated_2013-01.pgn.zst -o lichess_db.pgn

lichess_db_standard_rated_2013-01.pgn.zst: 92811021 bytes 


### Stockfish: https://stockfishchess.org/download/linux/

In [ ]:
!lscpu | grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz


In [ ]:
!wget --no-verbose https://stockfishchess.org/files/stockfish_15.1_linux_x64_avx2.zip

2023-03-21 22:07:40 URL:https://stockfishchess.org/files/stockfish_15.1_linux_x64_avx2.zip [27166402/27166402] -> "stockfish_15.1_linux_x64_avx2.zip" [1]


In [ ]:
!unzip stockfish_15.1_linux_x64_avx2.zip stockfish_15.1_linux_x64_avx2/stockfish-ubuntu-20.04-x86-64-avx2

Archive:  stockfish_15.1_linux_x64_avx2.zip
  inflating: stockfish_15.1_linux_x64_avx2/stockfish-ubuntu-20.04-x86-64-avx2  


### Python packages

In [ ]:
!pip install -q chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 KB 4.8 MB/s eta 0:00:00


In [ ]:
import chess
import chess.engine
import chess.pgn
import pandas as pd

from tqdm import tqdm

# Evaluation

In [ ]:
pgn = open("lichess_db.pgn")
engine = chess.engine.SimpleEngine.popen_uci("stockfish_15.1_linux_x64_avx2/stockfish-ubuntu-20.04-x86-64-avx2")

In [ ]:
game = chess.pgn.read_game(pgn)
board = game.board()

rows = []
depths = []

mate_score = 1000000

for move in tqdm(list(game.mainline_moves())):
    board.push(move)
    info = engine.analyse(board, chess.engine.Limit(time=1.0))
    depths.append(info['depth'])
    rows.append([board.fen(), info['score'].white().score(mate_score=mate_score)])

df = pd.DataFrame(rows, columns=['position', 'score'])

100%|██████████| 25/25 [00:23<00:00,  1.07it/s]


In [ ]:
print(f"min depth: {min(depths)}")
print(f"max depth: {max(depths)}")
print(f"avg depth: {sum(depths) / len(depths):.2f}")

min depth: 0
max depth: 245
avg depth: 27.64


In [ ]:
df.head(15)

,position,score
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,46
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,42
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,39
3,rnbqkbnr/p1pp1ppp/1p2p3/8/3PP3/8/PPP2PPP/RNBQK...,81
4,rnbqkbnr/p1pp1ppp/1p2p3/8/3PP3/P7/1PP2PPP/RNBQ...,77
5,rn1qkbnr/pbpp1ppp/1p2p3/8/3PP3/P7/1PP2PPP/RNBQ...,75
6,rn1qkbnr/pbpp1ppp/1p2p3/8/3PP3/P1N5/1PP2PPP/R1...,73
7,rn1qkb1r/pbpp1ppp/1p2p2n/8/3PP3/P1N5/1PP2PPP/R...,141
8,rn1qkb1r/pbpp1ppp/1p2p2B/8/3PP3/P1N5/1PP2PPP/R...,132
9,rn1qkb1r/pbpp1p1p/1p2p2p/8/3PP3/P1N5/1PP2PPP/R...,130


In [ ]:
engine.close()